In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

from IPython.display import clear_output
from sklearn import preprocessing

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# example of converting an image with the Keras API
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

from tensorflow.keras.initializers import glorot_uniform
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

/home/csgrads/samad028/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/csgrads/samad028/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/csgrads/samad028/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/csgrads/samad028

In [2]:
%run paths.py # Load all our paths
%run methods.py # Load all our methods

MODEL_FILE = 'model_cnn_data-1000000_epochs-100'
PREDICTION_MODEL = os.path.join(MODELS_DIR, MODEL_FILE+'.h5')

os.listdir(DATASET_DIR)

['test_images',
 'train_masks.zip',
 'train_images.zip',
 'car_models',
 '.ipynb_checkpoints',
 'car_models_json.zip',
 'car_models.zip',
 'test_masks',
 'train.csv',
 'train_images',
 'test_images.zip',
 'train_masks',
 'camera.zip',
 'sample_submission.csv',
 'car_models_json',
 'test_masks.zip',
 'camera']

In [3]:
df_train = pd.read_csv(os.path.join(DATASET_DIR, 'train.csv'))
df_train['NumCars'] = [int((x.count(' ')+1)/7) for x in df_train['PredictionString']]

In [4]:
model = tf.keras.models.load_model(PREDICTION_MODEL,custom_objects={'GlorotUniform': glorot_uniform()})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
testImageIds = os.listdir(TEST_IMAGES_DIR)

PredictionString = dict()
detection_probability = 50
totalFiles = len(testImageIds)


for count,ImageIdWithJPG in enumerate(testImageIds):
    
    ImageId = ImageIdWithJPG.replace(".jpg","")
    
    image = return_img(imageId=ImageId,datasetType='test')
    
    returned_results = detector.detectCustomObjectsFromImage(
        custom_objects=detector.CustomObjects(car=True),
        input_image=image, 
        input_type="array",
        output_image_path=os.path.join(DATASET_DIR , "detections/test/"+ImageId+".jpg"),
        minimum_percentage_probability=detection_probability,
        extract_detected_objects=True)

    detections, objects_path = returned_results
    
    PredictionString[ImageId] = ""
    
    eachCarString = []
    
    for eachObject, eachObjectPath in zip(detections, objects_path):
        ImageIdSubId = ImageId+"-"+str(eachObjectPath.split("objects/")[1].split(".")[0].split("-")[1])
        
        det = eachObject["box_points"]
        x1, y1, x2, y2 = det[0], det[1], det[2], det[3]
        extra_inputs = [x1, y1, x2, y2]
        temp_array = format_inputs_appended_with_image(ImageIdSubId,extra_inputs,'test')
        temp_array = np.array([temp_array])
        
        confidence = eachObject["percentage_probability"]/100
        
        xyz_ypr = model.predict(temp_array).flatten()
        
        x,y,z,yaw,pitch,roll = xyz_ypr[0],xyz_ypr[1],xyz_ypr[2],xyz_ypr[3],xyz_ypr[4],xyz_ypr[5]
        
        output_format_list = [yaw, pitch, roll, x, y, z, confidence]
        
        # Round off 2 decimal places
        output_format_list = [ round(elem, 2) for elem in output_format_list ]
        
        output_format_str = " ".join(map(str,output_format_list))
        
        eachCarString.append(output_format_str)
        #print(output_format_str)
        
    PredictionString[ImageId] = " ".join(eachCarString)
    
    #print(PredictionString[ImageId])
    
    if count%5==0:
        clear_output(wait=True)
        print(f"{count+1}/{totalFiles} Completed. Just tested Image: {ImageId}")
    
    """if count>5:
        break"""

2021/2021 Completed. Just tested Image: ID_bcadc06b8


In [17]:
submission = []
for key,value in PredictionString.items():
    sub=dict()
    sub["ImageId"] = key
    sub["PredictionString"] = value
    submission.append(sub)
    
df_submission = pd.DataFrame(submission)
df_submission.to_csv(os.path.join(SUBMISSION_DIR, f"submission_rows-{count}_model-{MODEL_FILE}.csv"),index=False)
display(df_submission)

,ImageId,PredictionString
0,ID_37377a64b,0.12 1.69 -2.4 -3.76 11.28 64.17 0.57 0.15 -0....
1,ID_fff387961,0.1 -1.94 -6.27 2.08 11.8 56.43 0.65 0.15 0.93...
2,ID_17b7c91d1,0.16 0.4 -3.46 -23.23 6.64 41.07 0.57 0.16 0.0...
3,ID_4c1863a6b,0.18 -0.17 -2.08 2.31 3.73 15.29 0.99
4,ID_4811592fb,0.15 1.37 -1.72 6.38 10.21 51.74 0.99 0.15 -0....
...,...,...
2016,ID_6cf4fe617,0.18 -0.67 -2.21 -4.89 6.56 34.97 0.58 0.18 -0...
2017,ID_9f7897f56,0.15 -0.9 -1.46 -12.57 7.4 41.52 0.89 0.15 1.0...
2018,ID_bf9c30a62,0.2 6.41 14.27 -25.42 5.76 27.85 0.73 0.17 1.3...
2019,ID_0a5908893,0.13 0.12 0.93 -8.49 16.32 87.7 0.51 0.14 0.96...
